In [1]:
import json
from pprint import pprint

In [2]:
with open("outputs/final_results.txt", "r") as f:
    final_results = json.loads(f.read())

all_results = {
    "baseline": {
        "kmedian": {
            "Synthetic": {
                "Objective": 299.859,
                "fairness error": 0.00,
                "balance": 1.00,
            },
            "Synthetic-unequal": {
                "Objective": 185.47,
                "fairness error": 0.77,
                "balance": 0.21,
            },
            "Adult": {
                "Objective": 19330.93,
                "fairness error": 0.27,
                "balance": 0.31,
            },
            "Bank": {
                "Objective": "N/A",
                "fairness error": "N/A",
                "balance": "N/A",
            },
            "CensusII": {
                "Objective": 2385997.92,
                "fairness error": 0.41,
                "balance": 0.38,
            },
        },
        "kmeans": {
            "Synthetic": {
                "Objective": 758.43,
                "fairness error": 0.00,
                "balance": 1.00,
            },
            "Synthetic-unequal": {
                "Objective": 180.00,
                "fairness error": 0.00,
                "balance": 0.33,
            },
            "Adult": {
                "Objective": 10913.84,
                "fairness error": 0.018,
                "balance": 0.41,
            },
            "Bank": {
                "Objective": 11331.51,
                "fairness error": 0.03,
                "balance": 0.16,
            },
            "CensusII": {
                "Objective": 1355457.02,
                "fairness error": 0.07,
                "balance": 0.77,
            },
        },
        "ncut": {
            "Synthetic": {
                "Objective": 0.0,
                "fairness error": 0.00,
                "balance": 1.00,
            },
            "Synthetic-unequal": {
                "Objective": 0.03,
                "fairness error": 0.00,
                "balance": 0.33,
            },
            "Adult": {
                "Objective": 0.47,
                "fairness error": 0.06,
                "balance": 0.32,
            },
            "Bank": {
                "Objective": "N/A",
                "fairness error": "N/A",
                "balance": "N/A",
            },
            "CensusII": {
                "Objective": "N/A",
                "fairness error": "N/A",
                "balance": "N/A",
            },
        },
    },
    "original": {
        "kmedian": {
            "Synthetic": {
                "Objective": 292.4,
                "fairness error": 0.00,
                "balance": 1.00,
            },
            "Synthetic-unequal": {
                "Objective": 174.81,
                "fairness error": 0.00,
                "balance": 0.33,
            },
            "Adult": {
                "Objective": 18467.75,
                "fairness error": 0.01,
                "balance": 0.43,
            },
            "Bank": {
                "Objective": 19527.08,
                "fairness error": 0.02,
                "balance": 0.18,
            },
            "CensusII": {
                "Objective": 1754109.46,
                "fairness error": 0.02,
                "balance": 0.78,
            },
        },
        "kmeans": {
            "Synthetic": {
                "Objective": 207.80,
                "fairness error": 0.00,
                "balance": 1.00,
            },
            "Synthetic-unequal": {
                "Objective": 159.75,
                "fairness error": 0.00,
                "balance": 0.33,
            },
            "Adult": {
                "Objective": 9984.01,
                "fairness error": 0.018,
                "balance": 0.41,
            },
            "Bank": {
                "Objective": 9392.20,
                "fairness error": 0.05,
                "balance": 0.17,
            },
            "CensusII": {
                "Objective": 1018996.53,
                "fairness error": 0.02,
                "balance": 0.78,
            },
        },
        "ncut": {
            "Synthetic": {
                "Objective": 0.0,
                "fairness error": 0.0,
                "balance": 1.00,
            },
            "Synthetic-unequal": {
                "Objective": 0.06,
                "fairness error": 0.00,
                "balance": 0.33,
            },
            "Adult": {
                "Objective": 0.74,
                "fairness error": 0.08,
                "balance": 0.30,
            },
            "Bank": {
                "Objective": 0.58,
                "fairness error": 0.39,
                "balance": 0.14,
            },
            "CensusII": {
                "Objective": 0.52,
                "fairness error": 0.41,
                "balance": 0.43,
            },
        },
    },
    "reproduction": final_results["reprod"],
    "lmbda tuned": final_results["lmbda_reprod"],
    "Lipschitz test": final_results["Lipz_replic"],
}
for source in all_results:
    for alg in all_results[source]:
        for dataset in all_results[source][alg]:
            for metric in all_results[source][alg][dataset]:
                value = all_results[source][alg][dataset][metric]
                if not isinstance(value, dict):
                    all_results[source][alg][dataset][metric] = {
                        "mean": value,
                        "std": None,
                    }
DATASETS = all_results["baseline"]["kmeans"].keys()
ALGS = all_results["baseline"].keys()
METRICS = all_results["baseline"]["kmeans"]["Synthetic"].keys()
SOURCES = all_results.keys()

ALG_TITLES = {
    "kmeans": r"\textbf{Fair $\boldsymbol{K}$-means}",
    "kmedian": r"\textbf{Fair $\boldsymbol{K}$-medians}",
    "ncut": r"\textbf{Fair Ncut}",
}

In [3]:
pprint(all_results)

{'Lipschitz test': {'kmeans': {'Adult': {'Objective': {'mean': 10103.201000000001,
                                                       'std': 130.59812630993855},
                                         'balance': {'mean': 0.4010000000000001,
                                                     'std': 0.002999999999999986},
                                         'fairness error': {'mean': 0.013533333333333336,
                                                            'std': 0.0005617433182117576},
                                         'time': {'mean': 43.023333333333326,
                                                  'std': 20.498222416156533}},
                               'Bank': {'Objective': {'mean': 9533.586000000001,
                                                      'std': 188.531548829367},
                                        'balance': {'mean': 0.16999999999999998,
                                                    'std': 2.7755575615628914e-17},
      

In [4]:
def fill_template(
    content, 
    column_count=5,
    title=r"\textbf{Fair $\boldsymbol{K}$-medians}",
    source_header=r"& Backurs et al. & VFC & Backurs et al. & VFC \\",
    caption=r"Comparison of the proposed Fair $K$-medians to \citep{backurs2019scalable}",
    label="tab:comparisonkmedian"
    ):
    layout = f"|l|{'|'.join('c'*(column_count-1))}|"
    return r"""
\begin{table}[]
    \centering
    \begin{tabular}{"""+ layout +r"""}
        \hline
            & \multicolumn{"""+ f"{column_count-1}" +r"""}{c|}{"""+ title +r"""} \\
            \cline{2-"""+ f"{column_count}" +r"""}
        \textbf{Datasets} & \multicolumn{"""+ f"{int((column_count-1)/2)}" +r"""}{c|}{Objective} &   \multicolumn{"""+ f"{int((column_count-1)/2)}" +r"""}{c|}{Fairness error / Balance} \\ \cline{2-"""+ f"{column_count}" +r"""}
"""+ source_header +r"""
        \hline
        """+ content +r"""
        \hline
    \end{tabular}
    \caption{"""+ caption +r"""}
    \label{"""+ label +r"""}
\end{table}
"""

In [5]:
def bold_best(values, lower_is_better=True):
    num_means = [v["mean"] for v in values if str(v["mean"]).replace('.', '').isnumeric()]
    best = max(num_means)
    if lower_is_better:
        best = min(num_means)

    str_values = []
    for v in values:
        mean = v["mean"]
        std = v["std"]

        if not isinstance(mean, str):
            mean = round(mean, 2)
        str_value = str(mean)
        if std != None:
            str_value += r"($\pm "+f"{std:.2f}$)"
        if mean == round(best,2):
            str_value = r"\textbf{"+str_value+"}"
        str_values.append(str_value)

    return str_values

def red_not_reprod(values, lower_is_better=True):
    values[1]["mean"] = round(values[1]["mean"], 2)
    original = values[0]
    reprod = values[1]
    difference = original["mean"] - reprod["mean"]
    if not lower_is_better:
        difference *= -1
    color = "red"
    if difference >= -0.01*abs(reprod["mean"]):
        color = None
    elif difference >= -reprod["std"]:
        color = "orange"

    str_values = []
    for i, v in enumerate(values):
        mean = v["mean"]
        std = v["std"]
        str_value = mean
        if not isinstance(mean, str):
            str_value = f"{mean:.2f}"
        if std != None:
            str_value += r"($\pm "+f"{std:.2f}$)"
        if i == 0 and color:
            str_value = r"\color{"+color+"}"+str_value+r"\color{black}"
        str_values.append(str_value)

    return str_values



sources = ["baseline", "original", "reproduction"]
sources = ["original", "reproduction"]
sources = ["baseline", "reproduction"]
sources = ["baseline", "lmbda tuned"]
sources = ["lmbda tuned all", "Lipschitz test"]
for alg in ALGS:
    print()
    rows = []
    for dataset in DATASETS:
        try:
            [all_results[source][alg][dataset] for source in sources if source != "lmbda tuned all"]
        except KeyError:
            continue

        results_by_metric = {}
        for metric in METRICS:
            values = []
            for source in sources:
                if source == "lmbda tuned all":
                    try:
                        values.append(all_results["lmbda tuned"][alg][dataset][metric])
                        continue
                    except KeyError:
                        source = "reproduction"
                values.append(all_results[source][alg][dataset][metric])

            results_by_metric[metric] = bold_best(values, metric != "balance")
            # results_by_metric[metric] = red_not_reprod(values, metric != "balance")
        columns = []
        columns.extend(results_by_metric["Objective"])
        columns.extend([f"{f} / {b}" for f, b in zip(results_by_metric["fairness error"], results_by_metric["balance"])])
        columns.insert(0, dataset)
        rows.append(" & ".join(columns)+r" \\")
    content = "\n".join(rows)
    title = ALG_TITLES[alg]

    source_header = "& "+" & ".join(sources*2)+r" \\"
    caption = "WRITE CAPTION"
    label = f"tab:comparison_{'VS'.join(sources)}_{alg}"
    print(fill_template(content, len(columns), title, source_header, caption, label))



\begin{table}[]
    \centering
    \begin{tabular}{|l|c|c|c|c|}
        \hline
            & \multicolumn{4}{c|}{\textbf{Fair $\boldsymbol{K}$-medians}} \\
            \cline{2-5}
        \textbf{Datasets} & \multicolumn{2}{c|}{Objective} &   \multicolumn{2}{c|}{Fairness error / Balance} \\ \cline{2-5}
& lmbda tuned all & Lipschitz test & lmbda tuned all & Lipschitz test \\
        \hline
        Adult & 17887.87($\pm 307.59$) & \textbf{17513.1($\pm 182.47$)} & \textbf{0.01($\pm 0.00$)} / \textbf{0.42($\pm 0.01$)} & \textbf{0.01($\pm 0.00$)} / 0.4($\pm 0.00$) \\
Bank & 20242.38($\pm 403.62$) & \textbf{19743.99($\pm 341.91$)} & \textbf{0.04($\pm 0.00$)} / \textbf{0.17($\pm 0.00$)} & 0.05($\pm 0.01$) / \textbf{0.17($\pm 0.00$)} \\
        \hline
    \end{tabular}
    \caption{WRITE CAPTION}
    \label{tab:comparison_lmbda tuned allVSLipschitz test_kmedian}
\end{table}



\begin{table}[]
    \centering
    \begin{tabular}{|l|c|c|c|c|}
        \hline
            & \multicolumn{4}{c|}{\te

In [ ]:
table = r"""
\begin{table}[]
    \centering
    \begin{tabular}{|l|c|c|c|c|}
        \hline
         & \multicolumn{4}{c|}{\textbf{Fair $\boldsymbol{K}$-medians}} \\
         \cline{2-5}
        \textbf{Datasets} & \multicolumn{2}{c|}{Objective} &   \multicolumn{2}{c|}{Fairness error / Balance} \\ \cline{2-5}
        & Backurs et al. & VFC & Backurs et al. & VFC \\
        \hline 
        Synthetic & 299.859 & \textbf{289.08 ($\mathbf{\pm 2.03}$)} & \textbf{0.00/1.00} & 0.82 ($\pm 1.05$)/0.34 ($\pm 0.21$) \\
        Synthetic-unequal & 185.47 & \textbf{174.82 ($\mathbf{\pm 0.00}$)} & 0.77/0.21 & \textbf{0.00 ($\mathbf{\pm 0.00}$)/0.33 ($\mathbf{\pm 0.00}$)} \\
        Adult & 19330.93 & \textbf{17763.41 ($\mathbf{\pm 244.70}$)} & 0.27/0.31 & \textbf{0.01 ($\mathbf{\pm 0.00}$)/0.42 ($\mathbf{\pm 0.01}$)} \\
        Bank & N/A & \textbf{20087.08 ($\mathbf{\pm 264.69}$)} & N/A &\textbf{0.04 ($\mathbf{\pm 0.01}$)/0.17 ($\mathbf{\pm 0.00}$)} \\
        CensusII & 2385997.92 & \textbf{1746850.32 ($\mathbf{\pm 11858.69}$)} & 0.41/0.38 & \textbf{0.02 ($\mathbf{\pm 0.00}$)/0.75 ($\mathbf{\pm 0.04}$)} \\
        Student & N/A & \textbf{5} & N/A &\textbf{ 5} \\
        \hline
    \end{tabular}
    \caption{Comparison of the proposed Fair $K$-medians to \citep{backurs2019scalable}}
    \label{tab:comparisonkmedian}
\end{table}
"""